In [1]:
from datasets import load_dataset, load_from_disk, Dataset, DatasetDict, concatenate_datasets
import numpy as np
import pandas as pd

In [3]:
fake_train_dataset = load_from_disk("fake_true_dataset")

In [4]:
fake_train_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
})

In [5]:
fake_train_dataset['train'][0]

{'text': 'Nuclear explosions release more energy than anything else that humans do. Luckily, the energy released by a nuclear blast is only about one trillionth of the power of the Earth spinning. Scientists compare it to trying to slow the speed of a truck by crashing it into a mosquito.',
 'label': 0}

In [11]:
fake_train_dataset['train'][6]

{'text': "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nContext:  \n Name some famous actors in the 1940's<|im_end|>\n<|im_start|>assistant\nSome famous actors in the 1940s were:\n\n1. Clark Gable\n2. James Caan\n3. James Caan\n4. Jack Armstrong\n5. James Caan\n6. Clark Gable\n7. James Caan\n8. Robert Taylor\n9. Robert Taylor\n10. James Caan<|im_end|>",
 'label': 1}